In [1]:
%autosave 10
import time

Autosaving every 10 seconds


In [2]:
import openpyxl
import pandas as pd

# Please ensure that the Length of the Final Cuts are rounded off considerably

In case of mismatch due to algorithm rounding off after the Pattern Generation sheet is obtained, it may be manually matched

In [3]:
DashBoard_Obj=pd.read_excel("DashBoard.xlsx","Objective",index_col=0)
print(DashBoard_Obj)

                                                    Value  \
Parameter                                                   
Problem                                                20   
Objective                                               4   
Number of Combinations to be Generated                 55   
Timelimit for each of the smallest problems            30   
Timelimit for the Large CSP problems                  500   
Maximum Number of Unique Standard Length usages         2   
Maximum Number of Unique Pattern usages per Sta...     50   

                                                                                              Remarks  
Parameter                                                                                              
Problem                                                                  Diameter of the Bars (in mm)  
Objective                                           Case Type 4 = Generating the Top Minimum Waste...  
Number of Combinations to be Gener

In [4]:
# Create a dictionary from the index and a specific column
Decisions = DashBoard_Obj['Value'].to_dict()
print(Decisions)
# Decisions["Maximum Number of (Unique) Standard Length usages"]

{'Problem': 20, 'Objective': 4, 'Number of Combinations to be Generated': 55, 'Timelimit for each of the smallest problems': 30, 'Timelimit for the Large CSP problems': 500, 'Maximum Number of Unique Standard Length usages': 2, 'Maximum Number of Unique Pattern usages per Standard Length': 50}


In [5]:
DashBoard_Dataset_Cuts=pd.read_excel("DashBoard.xlsx","Dataset_Cuts",index_col=0)
print(DashBoard_Dataset_Cuts)

                         Total Requirements
Lengths of Cuts Allowed                    
1.810                                     8
2.930                                  1443
3.030                                   332
3.893                                    24
3.951                                    48
4.046                                    24
4.332                                     8
4.345                                    24
4.503                                    24
5.755                                    14
6.492                                    16
6.573                                    32
7.090                                     4
7.337                                    32
7.390                                    16
7.676                                    24


In [6]:
DashBoard_Dataset_StandardLengths=pd.read_excel("DashBoard.xlsx","Dataset_StandardLengths",index_col=0)
#print(type(DashBoard_Dataset_StandardLengths))
print(DashBoard_Dataset_StandardLengths)

                                Associated Cost
Total Standard Lengths Allowed                 
8.0                                    0.251327
9.5                                    0.298451
11.0                                   0.345575


In [7]:
length_of_cuts_allowed = DashBoard_Dataset_Cuts.index.tolist()
print(length_of_cuts_allowed)

[1.81, 2.93, 3.03, 3.893, 3.951, 4.046, 4.332, 4.345, 4.503, 5.755, 6.492, 6.573, 7.09, 7.337, 7.39, 7.676]


In [8]:
requirements = DashBoard_Dataset_Cuts["Total Requirements"]
print(requirements)
#requirements[1.2]

Lengths of Cuts Allowed
1.810       8
2.930    1443
3.030     332
3.893      24
3.951      48
4.046      24
4.332       8
4.345      24
4.503      24
5.755      14
6.492      16
6.573      32
7.090       4
7.337      32
7.390      16
7.676      24
Name: Total Requirements, dtype: int64


In [9]:
total_max_standardlengths_allowed = DashBoard_Dataset_StandardLengths.index.tolist()
print(total_max_standardlengths_allowed)

[8.0, 9.5, 11.0]


In [10]:
combinations = range(1,int(Decisions["Number of Combinations to be Generated"])+1)
print(combinations)

range(1, 56)


In [11]:
#max_time_in_seconds = Decisions["Timelimit for each of the smallest problems"]

#Timelimit_for_each_final_CSP_problem_minimizing_WCSP = Decisions["Timelimit for the Large CSP problems"]

# Generating Combinations Depending on Objectives

In [12]:
import gurobipy as gp
from gurobipy import GRB

In [13]:
start_time_is_this=time.time()
print(start_time_is_this)

1702801365.1397781


# Constraints dependant on Objectives

In [14]:
if Decisions["Objective"] == 4:
    M = 10000000
    X_Standards = {}
    
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        
        for c in combinations:
            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                if cc < c:
                    for l in length_of_cuts_allowed:
                        m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                        g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                        prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                        prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                        prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                        prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                        prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                        prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                    prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            waste_generated = prob.objVal

            for l in length_of_cuts_allowed:
                X_store[c, l] = x[l].x
                if X_store[c, l] > 0:
                    print("\t X_store[", c, ",", l, "] = ", X_store[c, l])

            print("\t \t Total Waste Generated: ", waste_generated)
            X_store[c, "Waste Objective"] = waste_generated
            print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem of Standard length: ", max_length)

        X_Standards[max_length] = X_store.copy()

Max Standard Length Allowed:  8.0
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-10
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 16 columns and 16 nonzeros
Model fingerprint: 0xd09fbfec
Variable types: 0 continuous, 16 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 8e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 8e+00]
Found heuristic solution: objective 0.7600000
Presolve removed 0 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 11 columns, 11 nonzeros
Variable types: 0 continuous, 11 integer (6 binary)
Found heuristic solution: objective 0.0350000

Root relaxation: objective 0.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Cur

     0     0    0.00000    0    1    0.06100    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.06100    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.06100    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.06100    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.06100    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.06100    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.06100    0.00000   100%     -    0s
     0     2    0.00000    0    5    0.06100    0.00000   100%     -    0s

Cutting planes:
  MIR: 1
  StrongCG: 1

Explored 34 nodes (67 simplex iterations) in 0.06 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 0.061 0.429 0.76 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.100000000000e-02, best bound 6.100000000000e-02, gap 0.0000%
	 X_store[ 4 , 3.893 ] =  1.0
	 X_store[ 4 , 4.046 ] =  1.0
	 	 Total Waste 

Thread count was 4 (of 4 available processors)

Solution count 5: 0.156 0.214 0.23 ... 0.429

Optimal solution found (tolerance 1.00e-04)
Best objective 1.560000000000e-01, best bound 1.560000000000e-01, gap 0.0000%
	 X_store[ 7 , 3.893 ] =  1.0
	 X_store[ 7 , 3.951 ] =  1.0
	 	 Total Waste Generated:  0.15600000000000058
	 	 Total Time Taken:  0.1  seconds, for this smallest problem of Standard length:  8.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 680 rows, 240 columns and 1696 nonzeros
Model fingerprint: 0xc5d1c9d2
Variable types: 112 continuous, 128 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 630 rows and 215 columns
Presolve time: 0

  RHS range        [1e+00, 1e+07]
Presolve removed 901 rows and 305 columns
Presolve time: 0.01s
Presolved: 70 rows, 31 columns, 255 nonzeros
Variable types: 0 continuous, 31 integer (22 binary)
Found heuristic solution: objective 0.4870000
Found heuristic solution: objective 0.3340000

Root relaxation: objective 0.000000e+00, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    0.33400    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.33400    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.33400    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.33400    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.33400    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.33400    0.00000   100%     -    0s
     0     0    0.00000


Optimal solution found (tolerance 1.00e-04)
Best objective 4.350000000000e-01, best bound 4.350000000000e-01, gap 0.0000%
	 X_store[ 14 , 1.81 ] =  1.0
	 X_store[ 14 , 5.755 ] =  1.0
	 	 Total Waste Generated:  0.4350000000000005
	 	 Total Time Taken:  0.1  seconds, for this smallest problem of Standard length:  8.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1359 rows, 464 columns and 3376 nonzeros
Model fingerprint: 0x02bc601d
Variable types: 224 continuous, 240 integer (224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 1255 rows and 423 columns
Presolve time: 0.02s
Presolved: 104 rows, 41 columns, 372 nonzeros
Variable types: 0 continuous, 41 integer

...

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 3468 rows and 1156 columns
Presolve time: 0.06s
Presolved: 510 rows, 172 columns, 1845 nonzeros
Variable types: 0 continuous, 172 integer (118 binary)
Found heuristic solution: objective 1.9500000
Found heuristic solution: objective 1.5140000

Root relaxation: objective 0.000000e+00, 123 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    5    1.51400    0.00000   100%     -    0s
H    0     0                       0.9130000    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.91300    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.91300    0.00000   100%     -    0s
H    0     0                       0.7420000    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.74200    0.00000 

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4269 rows, 1424 columns and 10576 nonzeros
Model fingerprint: 0x33978641
Variable types: 704 continuous, 720 integer (704 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 3720 rows and 1240 columns
Presolve time: 0.04s
Presolved: 549 rows, 184 columns, 1982 nonzeros
Variable types: 0 continuous, 184 integer (126 binary)
Found heuristic solution: objective 0.9520000
Found heuristic solution: objective 0.7990000

Root relaxation: objective 0.000000e+00, 137 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    3    0.79900    0.00000   100%     -    0s
     0     0    0.00000    0    3  

Variable types: 0 continuous, 194 integer (133 binary)
Found heuristic solution: objective 1.6570000

Root relaxation: objective -1.776357e-15, 150 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.00000    0   13    1.65700   -0.00000   100%     -    0s
H    0     0                       0.8070000   -0.00000   100%     -    0s
H    0     0                       0.7360000   -0.00000   100%     -    0s
     0     0    0.00000    0    6    0.73600    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.73600    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.73600    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.73600    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.73600    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.73600    0.00000  


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8    1.51400    0.00000   100%     -    0s
H    0     0                       1.1990000    0.00000   100%     -    0s
     0     0    0.00000    0   15    1.19900    0.00000   100%     -    0s
H    0     0                       0.6150000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.61500    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.61500    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.61500    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.61500    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.61500    0.00000   100%     -    0s
     0     2    0.00000    0    1    0.61500    0.00000   100%     -    0s
*   74    21              13       0.6080000    0.00000   100%  15.2    0s
*   82    21           


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6    1.04700    0.00000   100%     -    0s
     0     0    0.00000    0    6    1.04700    0.00000   100%     -    0s
     0     0    0.00000    0    6    1.04700    0.00000   100%     -    0s
     0     0    0.00000    0    6    1.04700    0.00000   100%     -    0s
     0     0    0.00000    0    6    1.04700    0.00000   100%     -    0s
     0     0    0.00000    0    6    1.04700    0.00000   100%     -    0s
H    0     0                       0.9130000    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.91300    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.91300    0.00000   100%     -    0s
     0     2    0.00000    0    6    0.91300    0.00000   100%     -    0s
*   13    13               4       0.6410000    0.00000   100%  13.5    0s
H   62    23           

In [15]:
max_combination_for_each_standard = {}

if Decisions["Objective"] == 4:
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        combination_previous_end_number = Decisions["Number of Combinations to be Generated"]
        current_combination_number = combination_previous_end_number

        for ll in length_of_cuts_allowed:
            if ll > max_length:
                continue

            current_combination_number += 1

            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 0
            prob.addConstr(x[ll] >= 1)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_Standards[max_length][cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_Standards[max_length][cc, l])

                    prob.addConstr(x[l] - X_Standards[max_length][cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_Standards[max_length][cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_Standards[max_length][cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_Standards[max_length][cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            for cc in range(combination_previous_end_number + 1, current_combination_number):
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                    prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            
            
            
            if prob.SolCount:
                waste_generated = prob.objVal

                for l in length_of_cuts_allowed:
                    X_store[current_combination_number, l] = x[l].x
                    if X_store[current_combination_number, l] > 0:
                        print("\t X_store[", current_combination_number, ",", l, "] = ", X_store[current_combination_number, l])

                print("\t \t Total Waste Generated: ", waste_generated,
                      " during mandatory unique usage of cut-length: ", ll, "in standard length: ", max_length)
                X_store[current_combination_number, "Waste Objective"] = waste_generated
                print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem")
            else:
                current_combination_number-=1

        max_combination_for_each_standard[max_length] = current_combination_number
        X_Standards[max_length].update(X_store)

Max Standard Length Allowed:  8.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5337 rows, 1776 columns and 13217 nonzeros
Model fingerprint: 0xadcd54de
Variable types: 880 continuous, 896 integer (880 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 5231 rows and 1741 columns
Presolve time: 0.02s
Presolved: 106 rows, 35 columns, 257 nonzeros
Variable types: 0 continuous, 35 integer (32 binary)

Root relaxation: objective 2.300000e-01, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.23000    0 

  RHS range        [1e+00, 1e+07]
Presolve removed 5570 rows and 1852 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5628 rows, 1872 columns and 13937 nonzeros
Model fingerprint: 0xfd892b1a
Variable types: 928 continuous, 944 integer (928 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 5570 rows and 1852 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was

Presolve time: 0.03s
Presolved: 198 rows, 68 columns, 614 nonzeros
Variable types: 0 continuous, 68 integer (55 binary)
Found heuristic solution: objective 2.8500000
Found heuristic solution: objective 1.8300000

Root relaxation: objective 0.000000e+00, 60 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   15    1.83000    0.00000   100%     -    0s
     0     0    0.00000    0   24    1.83000    0.00000   100%     -    0s
     0     0    0.00000    0    2    1.83000    0.00000   100%     -    0s
H    0     0                       1.7300000    0.00000   100%     -    0s
     0     0    0.01761    0   22    1.73000    0.01761  99.0%     -    0s
     0     0    0.02000    0    1    1.73000    0.02000  98.8%     -    0s
     0     0    0.02016    0   28    1.73000    0.02016  98.8%     -    0s
H    0     0           

     0     0    1.68000    0    7    1.96700    1.68000  14.6%     -    0s

Cutting planes:
  Learned: 9
  Cover: 1
  Implied bound: 1
  MIR: 15
  RLT: 1
  Relax-and-lift: 1

Explored 1 nodes (135 simplex iterations) in 0.08 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 1.967 2.85 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.967000000000e+00, best bound 1.967000000000e+00, gap 0.0000%
	 X_store[ 58 , 3.03 ] =  1.0
	 X_store[ 58 , 4.503 ] =  1.0
	 	 Total Waste Generated:  1.9670000000000005  during mandatory unique usage of cut-length:  3.03 in standard length:  9.5
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5628 rows, 1872 columns and 13937 nonzeros
Model fingerpr

  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 6113 rows and 2032 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 2.067 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.067000000000e+00, best bound 2.067000000000e+00, gap 0.0000%
	 X_store[ 64 , 2.93 ] =  1.0
	 X_store[ 64 , 4.503 ] =  1.0
	 	 Total Waste Generated:  2.067  during mandatory unique usage of cut-length:  4.503 in standard length:  9.5
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6210 rows, 2064 columns and 15377 nonzeros
M

Thread count was 1 (of 4 available processors)

Solution count 1: 1.824 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.824000000000e+00, best bound 1.824000000000e+00, gap 0.0000%
	 X_store[ 71 , 7.676 ] =  1.0
	 	 Total Waste Generated:  1.8239999999999998  during mandatory unique usage of cut-length:  7.676 in standard length:  9.5
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Max Standard Length Allowed:  11.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5337 rows, 1776 columns and 13217 nonzeros
Model fingerprint: 0xa770019c
Variable types: 880 continuous, 896 integer (880 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 

  MIR: 3
  RLT: 5

Explored 68 nodes (470 simplex iterations) in 0.13 seconds (0.04 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 0.695 0.73 1.42 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.950000000000e-01, best bound 6.950000000000e-01, gap 0.0000%
	 X_store[ 58 , 2.93 ] =  1.0
	 X_store[ 58 , 3.03 ] =  1.0
	 X_store[ 58 , 4.345 ] =  1.0
	 	 Total Waste Generated:  0.6950000000000003  during mandatory unique usage of cut-length:  3.03 in standard length:  11.0
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5628 rows, 1872 columns and 13937 nonzeros
Model fingerprint: 0x72b47fdb
Variable types: 928 continuous, 944 integer (928 binary)
Coefficient statistics:
  Matrix range     [1e


Root relaxation: objective 8.881784e-16, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    0.79900    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.79900    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.79900    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.79900    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.79900    0.00000   100%     -    0s
     0     0    0.00000    0   11    0.79900    0.00000   100%     -    0s
     0     0    0.03700    0   13    0.79900    0.03700  95.4%     -    0s
     0     0    0.03940    0   15    0.79900    0.03940  95.1%     -    0s
     0     0    0.03940    0   15    0.79900    0.03940  95.1%     -    0s
     0     0    0.04500    0   13    0.79900    0.04500  94.4%     -    0s
     0    


Root relaxation: objective 0.000000e+00, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   17    1.06700    0.00000   100%     -    0s
     0     0    0.00000    0    3    1.06700    0.00000   100%     -    0s
     0     0    0.12428    0    4    1.06700    0.12428  88.4%     -    0s
     0     0    0.12662    0    6    1.06700    0.12662  88.1%     -    0s
     0     0    0.12662    0    7    1.06700    0.12662  88.1%     -    0s
     0     0    0.18400    0    1    1.06700    0.18400  82.8%     -    0s
     0     0    0.18400    0   13    1.06700    0.18400  82.8%     -    0s
H    0     0                       0.7420000    0.18400  75.2%     -    0s
     0     0    0.27225    0   16    0.74200    0.27225  63.3%     -    0s
     0     0    0.27712    0   17    0.74200    0.27712  62.7%     -    0s
     0    

	 X_store[ 69 , 2.93 ] =  1.0
	 X_store[ 69 , 7.337 ] =  1.0
	 	 Total Waste Generated:  0.7330000000000005  during mandatory unique usage of cut-length:  7.337 in standard length:  11.0
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6695 rows, 2224 columns and 16577 nonzeros
Model fingerprint: 0xce168add
Variable types: 1104 continuous, 1120 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 6695 rows and 2224 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 4 available 

In [16]:
import winsound
winsound.Beep(500,1000)
max_combination_for_each_standard

{8.0: 58, 9.5: 71, 11.0: 71}

In [17]:
wb=openpyxl.Workbook()

sheet_for_Combinations = wb.active

row_number=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Length of Cut"

column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value="Combination "+str(column_number-1)
        column_number+=1




for l in length_of_cuts_allowed:
    row_number+=1
    sheet_for_Combinations.cell(row=row_number,column=1).value= l
    column_number=2
    for i,max_length in enumerate(total_max_standardlengths_allowed):
        #for c in range(1,current_combination_number+1):
        for c in range(1,max_combination_for_each_standard[max_length]+1):
            sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,l]
            column_number+=1
        
        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Standard Max Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= max_length
        column_number+=1
        

        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Wasted Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,"Waste Objective"]
        column_number+=1

In [18]:
#file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+str(max_time_in_seconds)+".xlsx"
file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+".xlsx"
wb.save(file_path)
print(file_path)

./Problem_20__CombinationGeneration_55__TimeLimit_.xlsx


In [19]:
X_Standards

{8.0: {(1, 1.81): 0.0,
  (1, 2.93): -0.0,
  (1, 3.03): -0.0,
  (1, 3.893): -0.0,
  (1, 3.951): 1.0,
  (1, 4.046): 1.0,
  (1, 4.332): -0.0,
  (1, 4.345): -0.0,
  (1, 4.503): -0.0,
  (1, 5.755): -0.0,
  (1, 6.492): -0.0,
  (1, 6.573): -0.0,
  (1, 7.09): -0.0,
  (1, 7.337): -0.0,
  (1, 7.39): -0.0,
  (1, 7.676): -0.0,
  (1, 'Waste Objective'): 0.0030000000000001137,
  (2, 1.81): 2.0,
  (2, 2.93): -0.0,
  (2, 3.03): -0.0,
  (2, 3.893): -0.0,
  (2, 3.951): -0.0,
  (2, 4.046): 0.0,
  (2, 4.332): -0.0,
  (2, 4.345): 1.0,
  (2, 4.503): -0.0,
  (2, 5.755): -0.0,
  (2, 6.492): 0.0,
  (2, 6.573): -0.0,
  (2, 7.09): 0.0,
  (2, 7.337): -0.0,
  (2, 7.39): 0.0,
  (2, 7.676): -0.0,
  (2, 'Waste Objective'): 0.03500000000000014,
  (3, 1.81): 2.0,
  (3, 2.93): -0.0,
  (3, 3.03): -0.0,
  (3, 3.893): -0.0,
  (3, 3.951): -0.0,
  (3, 4.046): 0.0,
  (3, 4.332): 1.0,
  (3, 4.345): 0.0,
  (3, 4.503): 0.0,
  (3, 5.755): 0.0,
  (3, 6.492): 0.0,
  (3, 6.573): 0.0,
  (3, 7.09): 0.0,
  (3, 7.337): 0.0,
  (3, 7.39):

In [20]:
for max_length in total_max_standardlengths_allowed:
    #print(max_length)
    print(X_Standards[max_length])

{(1, 1.81): 0.0, (1, 2.93): -0.0, (1, 3.03): -0.0, (1, 3.893): -0.0, (1, 3.951): 1.0, (1, 4.046): 1.0, (1, 4.332): -0.0, (1, 4.345): -0.0, (1, 4.503): -0.0, (1, 5.755): -0.0, (1, 6.492): -0.0, (1, 6.573): -0.0, (1, 7.09): -0.0, (1, 7.337): -0.0, (1, 7.39): -0.0, (1, 7.676): -0.0, (1, 'Waste Objective'): 0.0030000000000001137, (2, 1.81): 2.0, (2, 2.93): -0.0, (2, 3.03): -0.0, (2, 3.893): -0.0, (2, 3.951): -0.0, (2, 4.046): 0.0, (2, 4.332): -0.0, (2, 4.345): 1.0, (2, 4.503): -0.0, (2, 5.755): -0.0, (2, 6.492): 0.0, (2, 6.573): -0.0, (2, 7.09): 0.0, (2, 7.337): -0.0, (2, 7.39): 0.0, (2, 7.676): -0.0, (2, 'Waste Objective'): 0.03500000000000014, (3, 1.81): 2.0, (3, 2.93): -0.0, (3, 3.03): -0.0, (3, 3.893): -0.0, (3, 3.951): -0.0, (3, 4.046): 0.0, (3, 4.332): 1.0, (3, 4.345): 0.0, (3, 4.503): 0.0, (3, 5.755): 0.0, (3, 6.492): 0.0, (3, 6.573): 0.0, (3, 7.09): 0.0, (3, 7.337): 0.0, (3, 7.39): 0.0, (3, 7.676): 0.0, (3, 'Waste Objective'): 0.04800000000000004, (4, 1.81): 0.0, (4, 2.93): 0.0, (4

# Main CSProblem starts here:~

(designed to also run separately from the above program, be cautious about the intermediate file name changes if done manually)

In [21]:
import gurobipy as gp
from gurobipy import GRB

from math import ceil

In [22]:
patterns_df = pd.read_excel(file_path, index_col=0)
#print(patterns_df)

In [23]:
# Get the last two rows into a new DataFrame
last_two_rows = patterns_df.tail(2).copy()  # tail(2) gets the last 2 rows

# Remove the last two rows from the original DataFrame
new_df = patterns_df.drop(patterns_df.tail(2).index)

In [24]:
# Find unique elements in the first row (index 0)
unique_elements_row = last_two_rows.iloc[0].unique()

print("Unique elements in the first row of last_two_rows:")
print(unique_elements_row)

Unique elements in the first row of last_two_rows:
[ 8.   9.5 11. ]


In [25]:
all_length_of_cuts_allowed = new_df.index.tolist() # we could have used length_of_cuts_allowed, but here we take data from the Excel Sheet as we may want to change something for Later (in the Excel Sheet manually) which is however not recommended nor necessary
all_length_of_cuts_allowed

[1.81,
 2.93,
 3.03,
 3.893,
 3.951,
 4.046,
 4.332,
 4.345,
 4.503,
 5.755,
 6.492,
 6.573,
 7.09,
 7.337,
 7.39,
 7.676]

# Waste, Cost, Standard, Pattern: The WCSP sequence of Minimization (4 sequential problems

In [26]:
# Constraints
sl_key__value_c_mapping_dict={}
set_of_encountered_sl=set()
for c in patterns_df:
    sl = last_two_rows[c]["Standard Max Length"]
    if sl in set_of_encountered_sl:
        sl_key__value_c_mapping_dict[sl].add(c)
    else:
        sl_key__value_c_mapping_dict[sl]={c}
        set_of_encountered_sl.add(sl)
    
#print(sl_key__value_c_mapping_dict)

# WASTE MINIMIZATION

In [27]:
# Create a Gurobi model
CSProb = gp.Model("Stock Cutting with Waste Minimization")

# Variables
x = {}

for c in patterns_df:
    x[c] = CSProb.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{c}]')


# Objective: Minimize Waste
CSProb.setObjective(gp.quicksum(x[c] * last_two_rows[c]["Wasted Length"] for c in patterns_df), sense=GRB.MINIMIZE)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb.addConstr(gp.quicksum(x[c] * new_df[c][l] for c in patterns_df) >= requirements[l])





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb.Params.MIPFocus=2


# Optimize the model
CSProb.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 16 rows, 200 columns and 405 nonzeros
Model fingerprint: 0x7fda7340
Variable types: 0 continuous, 200 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [3e-03, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+03]
Found heuristic solution: objective 1533.9670000
Presolve

# COST MINIMIZATION

In [28]:
# Create a new Gurobi model
CSProb2 = gp.Model("Stock Cutting with Cost Minimization")

# Variables
x_2 = {}

for c in patterns_df:
    x_2[c] = CSProb2.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_2[{c}]')



# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb2.addConstr(gp.quicksum(x_2[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000, name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb2.addConstr(gp.quicksum(x_2[c] * new_df[c][l] for c in patterns_df) >= requirements[l])


# Objective 2
CSProb2.setObjective(gp.quicksum(x_2[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][last_two_rows[c]["Standard Max Length"]] for c in patterns_df),
                     sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb2.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb2.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb2.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb2.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb2.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb2.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb2.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb2.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb2.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb2.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb2.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb2.Params.MIPFocus=2


# Optimize the model
CSProb2.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 17 rows, 200 columns and 605 nonzeros
Model fingerprint: 0x3d1be9cb
Variable types: 0 continuous, 200 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 6e+00]
  Objective range  [3e-01, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.0

# STANDARDs MINIMIZATION

In [29]:
# STANDARDs MINIMIZATION
M=100000

# Create a new Gurobi model
CSProb3 = gp.Model("Stock Cutting with Unique Standard length usage minimization")

#Debugging Mode
CSProb3.setParam('OutputFlag', 1)

# Variables
x_3 = {}
b_3 = {}

for c in patterns_df:
    x_3[c] = CSProb3.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_3[{c}]')

for sl in unique_elements_row:
    b_3[float(sl)] = CSProb3.addVar(vtype=GRB.BINARY, name=f'b_3[{sl}]')

    
    
    
# Constraints

for sl in unique_elements_row:
    CSProb3.addConstr( gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_3[float(sl)])
    CSProb3.addConstr( M*b_3[float(sl)] >= gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    

    

# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000,name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb3.addConstr(gp.quicksum(x_3[c] * new_df[c][l] for c in patterns_df) >= requirements[l],name=f"Requirement const. for {l}")


# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000, name="Obj2 as Const.")


# New Objective 3 to Minimize the number of Unique Standard Length Usage
CSProb3.setObjective(gp.quicksum(b_3[float(sl)] for sl in unique_elements_row),sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb3.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb3.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb3.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb3.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb3.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb3.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb3.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb3.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb3.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb3.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb3.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb3.Params.MIPFocus=2

#CSProb3.Params.NumericFocus = 3  # Experiment with different values (0, 1, 2, 3)


# Optimize the model
CSProb3.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 24 rows, 203 columns and 1211 nonzeros
Model fingerprint: 0x8b33f48f
Variable types: 0 continuous, 203 integer (3 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+03]
Presolve added 1 rows and 0 columns
Presolve removed 0 r

In [30]:
# IMPORTANT

# #Identify Infeasible Constraints:
# #Use the infeasibility information (model.IIS) to identify the constraints causing infeasibility. This can be done as follows:

# CSProb3.computeIIS()
# print('IIS constraints:')
# for constr in CSProb3.getConstrs():
#     if constr.IISConstr:
#         print(constr.ConstrName)

In [31]:
# Print variable values
for var in CSProb3.getVars():
    print(f'{var.varName}: {var.x}')

# # Print constraint values
# for constr in CSProb3.getConstrs():
#     print(f'{constr.ConstrName}: {constr.Pi} (dual value)')

# Print objective value
print(f'Objective Value: {CSProb3.objVal}')

x_3[Combination 1]: 48.0
x_3[Combination 2]: -0.0
x_3[Combination 3]: -0.0
x_3[Combination 4]: -0.0
x_3[Combination 5]: -0.0
x_3[Combination 6]: -0.0
x_3[Combination 7]: -0.0
x_3[Combination 8]: -0.0
x_3[Combination 9]: -0.0
x_3[Combination 10]: -0.0
x_3[Combination 11]: -0.0
x_3[Combination 12]: -0.0
x_3[Combination 13]: -0.0
x_3[Combination 14]: -0.0
x_3[Combination 15]: -0.0
x_3[Combination 16]: -0.0
x_3[Combination 17]: -0.0
x_3[Combination 18]: -0.0
x_3[Combination 19]: -0.0
x_3[Combination 20]: -0.0
x_3[Combination 21]: -0.0
x_3[Combination 22]: -0.0
x_3[Combination 23]: -0.0
x_3[Combination 24]: -0.0
x_3[Combination 25]: -0.0
x_3[Combination 26]: -0.0
x_3[Combination 27]: -0.0
x_3[Combination 28]: -0.0
x_3[Combination 29]: -0.0
x_3[Combination 30]: -0.0
x_3[Combination 31]: -0.0
x_3[Combination 32]: -0.0
x_3[Combination 33]: -0.0
x_3[Combination 34]: -0.0
x_3[Combination 35]: -0.0
x_3[Combination 36]: -0.0
x_3[Combination 37]: -0.0
x_3[Combination 38]: -0.0
x_3[Combination 39]: 

# PATTERNs MINIMIZATION

In [32]:
# Create a new Gurobi model
CSProb4 = gp.Model("Stock Cutting with Minimum Unique Pattern usage")

# Variables
x_4 = {}
y_4 = {}
b_4 = {}

for c in patterns_df:
    x_4[c] = CSProb4.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_4[{c}]')
    y_4[c] = CSProb4.addVar(vtype=GRB.BINARY, name=f'y_4[{c}]')

for sl in unique_elements_row:
    b_4[sl] = CSProb4.addVar(vtype=GRB.BINARY, name=f'b_4[{sl}]')

# Constraints
for c in patterns_df:
    #CSProb4.addConstr(x_4[c] >= b_4[last_two_rows[c]["Standard Max Length"]]) #Wrong
    CSProb4.addConstr(x_4[c] >= y_4[c])
    #CSProb4.addConstr(M * b_4[last_two_rows[c]["Standard Max Length"]] >= x_4[c]) #Wrong
    CSProb4.addConstr(M * y_4[c] >= x_4[c])

    
# Constraints
for sl in unique_elements_row:
    CSProb4.addConstr( gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_4[float(sl)])
    CSProb4.addConstr( M*b_4[float(sl)] >= gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    
# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb4.addConstr(gp.quicksum(x_4[c] * new_df[c][l] for c in patterns_df) >= requirements[l])

# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000)




# Previous Objective 3: to Minimize the number of Unique Standard Length Usage
CSProb4.addConstr(gp.quicksum(b_4[sl] for sl in unique_elements_row) <= ceil(CSProb3.objVal *1000)/1000)


# New Objective 4 to Minimize the number of Unique Pattern Usage
CSProb4.setObjective(gp.quicksum(y_4[c] for c in patterns_df),sense=GRB.MINIMIZE)





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb4.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb4.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb4.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb4.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb4.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb4.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb4.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb4.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb4.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb4.Params.MIPGap = 0
print("Default MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb4.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb4.Params.MIPFocus=2


# Optimize the model
CSProb4.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Default MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 425 rows, 403 columns and 2014 nonzeros
Model fingerprint: 0xa97c59cd
Variable types: 0 continuous, 403 integer (203 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+03]
Presolve added 0 rows and 2 columns
Presolve time: 0.0

# Populate Result Excel Sheet

In [33]:
end_time_is_this=time.time()
print(end_time_is_this)

actual_time_taken = end_time_is_this - start_time_is_this
actual_time_taken

1702801428.7293496


63.589571475982666

In [34]:
winsound.Beep(500,1000)
wb_final=openpyxl.Workbook()

sheet_for_Usage = wb_final.active
sheet_for_Usage.title = 'Variable x and y'
sheet_for_b = wb_final.create_sheet(title='Variable b')
sheet_for_Objectives = wb_final.create_sheet(title='Objectives')

#sheet_for_x
row_number=1
sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage in Objective 1 (x)"
sheet_for_Usage.cell(row=row_number,column=4).value= "No. of Usage in Objective 2 (x_2)"
sheet_for_Usage.cell(row=row_number,column=5).value= "No. of Usage in Objective 3 (x_3)"
sheet_for_Usage.cell(row=row_number,column=6).value= "No. of Usage in Objective 4 (x_4)"
sheet_for_Usage.cell(row=row_number,column=7).value= "Unique Pattern Usages in Objective 4 (y_4)"

for c in patterns_df:
    row_number+=1
    sheet_for_Usage.cell(row=row_number,column=1).value=c
    sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
    sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
    sheet_for_Usage.cell(row=row_number,column=4).value=x_2[c].x
    sheet_for_Usage.cell(row=row_number,column=5).value=x_3[c].x
    sheet_for_Usage.cell(row=row_number,column=6).value=x_4[c].x
    sheet_for_Usage.cell(row=row_number,column=7).value=y_4[c].x
    #if x_3[c].x:
    #    print(c, " used " , x_3[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])


    
#sheet_for_b
row_number=1
sheet_for_b.cell(row=row_number,column=1).value= "Standard Length"
sheet_for_b.cell(row=row_number,column=2).value= "Utilization in Objective 3 (b-3)"
sheet_for_b.cell(row=row_number,column=3).value= "Utilization in Objective 4 (b_4)"

for sl in unique_elements_row:
    row_number+=1
    sheet_for_b.cell(row=row_number,column=1).value=sl
    sheet_for_b.cell(row=row_number,column=2).value=b_3[sl].x
    sheet_for_b.cell(row=row_number,column=3).value=b_4[sl].x
    
    
#sheet_for_Objectives
row_number=1
sheet_for_Objectives.cell(row=row_number,column=1).value= "Objective 1"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Objective 2"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Objective 3"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Objective 4"
sheet_for_Objectives.cell(row=row_number,column=5).value= "Total Time taken for the Pattern Generations and all Objective Computations"
row_number=2
sheet_for_Objectives.cell(row=row_number,column=1).value= "Minimum Total WASTE"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Minimum Total COST"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Minimum Unique STANDARD LENGTH Usage"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Minimum Unique PATTERN Usage"
sheet_for_Objectives.cell(row=row_number,column=5).value= "(Seconds)"
row_number=3
sheet_for_Objectives.cell(row=row_number,column=1).value= CSProb.objVal
sheet_for_Objectives.cell(row=row_number,column=2).value= CSProb2.objVal
sheet_for_Objectives.cell(row=row_number,column=3).value= CSProb3.objVal
sheet_for_Objectives.cell(row=row_number,column=4).value= CSProb4.objVal
sheet_for_Objectives.cell(row=row_number,column=5).value= actual_time_taken


Final_output_file_path = "./Prob"+str(Decisions["Problem"])+"_PatternUsage_with_WCSP_Minimization"+".xlsx"
wb_final.save(Final_output_file_path)

In [35]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
#     if x[c].x:
#         print(c, " used " , x[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])

        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Trim Loss"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb.objVal
# print("Objective Value:", CSProb.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumTrimLoss"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)

In [36]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x_2[c].x
#     if x_2[c].x:
#         print(c, " used " , x_2[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])
        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Total Cost"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb2.objVal
# print("Objective Value:", CSProb2.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumCost"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)